In [36]:
import pandas as pd
import sys
sys.path.append('..')
import db
import inspect
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import numpy as np
%matplotlib inline

In [37]:
conn=db.get_conn()
data_original=pd.read_sql(sql="select * from _201904 where monthly_salary>0 and monthly_salary<15", con=conn)
conn.close()

In [38]:
error_job_ids=['104660258','104142922','108434795','101357291','106253516','110368302','111391233','108665401','109277048'
                  ,'73857191','108584955','102824950','102824949','111391233','110884556']
data=data_original[~data_original.job_id.isin(error_job_ids)]

In [39]:
del data['publish_date']
del data['published_on_weekend']
del data['title']
del data['company_title']
del data['company_description']
del data['job_description']
del data['job_id']

In [40]:
def get_sub_stats(data, prefix):
    
    features = [feature for feature in data.columns if feature.startswith(prefix)]
    salary_mean=[]
    salary_median=[]
    count=[]
    for feature in features:
        salary_mean.append(data[data[feature]==1].monthly_salary.mean()*10000)
        salary_median.append(data[data[feature]==1].monthly_salary.median()*10000)
        count.append(data[data[feature]==1].shape[0])
    sub_data=pd.DataFrame()
    sub_data[prefix]=[f.replace(prefix,'') for f in features]
    sub_data['salary_mean']=salary_mean
    sub_data['salary_median']=salary_median
    sub_data['head_count']=count
    sub_data['percentage']=count/np.sum(count)
    sub_data=sub_data.sort_values(by='salary_mean', ascending=False)

    return sub_data

In [41]:
def add_category_column(data, prefix):
    prefix_=prefix+"_"
    features = [feature for feature in data.columns if feature.startswith(prefix)]
    data.loc[:,prefix]=['']*data.shape[0]
    for feature in features:
        data.loc[data[feature]==1,prefix]=feature.replace(prefix_,'')
    return data

In [42]:
data=add_category_column(data,'city')

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


# 城市 City

In [43]:
data_city=get_sub_stats(data,'city_')
data_city.style.format({"percentage":"{:.2%}","salary_mean":"{:.0f}","salary_median":"{:.0f}"})

,city_,salary_mean,salary_median,head_count,percentage
0,beijing,17453,15000,13823,9.90%
17,shanghai,16626,15000,25888,18.55%
19,shenzhen,15631,14000,20757,14.87%
9,hangzhou,14977,12500,8690,6.23%
8,guangzhou,13032,12000,17068,12.23%
14,nanjing,12874,12500,7091,5.08%
3,chengdu,11896,11500,7028,5.04%
6,dongguan,11353,10417,1632,1.17%
22,xian,11075,10000,3833,2.75%
21,wuhan,11074,10417,8027,5.75%


程序员收入最高的城市分别为：北京，上海，深圳，广州

In [44]:
translate_dict={}
translate_dict['beijing']='北京'
translate_dict['shanghai']='上海'
translate_dict['shenzhen']='深圳'
translate_dict['hangzhou']='杭州'
translate_dict['guangzhou']='广州'
translate_dict['nanjing']='南京'
translate_dict['chengdu']='成都'
translate_dict['dongguan']='东莞'
translate_dict['xian']='西安'
translate_dict['wuhan']='武汉'
translate_dict['tianjin']='天津'
translate_dict['changsha']='长沙'
translate_dict['ningbo']='宁波'
translate_dict['fuzhou']='福州'
translate_dict['dalian']='大连'
translate_dict['chongqing']='重庆'
translate_dict['qingdao']='青岛'
translate_dict['jinan']='济南'
translate_dict['hefei']='合肥'
translate_dict['changchun']='长春'
translate_dict['kuming']='昆明'
translate_dict['zhengzhou']='郑州'
translate_dict['shenyang']='沈阳'
translate_dict['harbin']='哈尔滨'


In [45]:
map_data_salary = []
map_data_headcount = []
for index , row in data_city.iterrows():
    map_data_salary.append((translate_dict[row[0]],row[1]))
    map_data_headcount.append((translate_dict[row[0]],row[3]))

In [60]:
from pyecharts import Map
job_map=Map()
attr, value = geo.cast(map_data)
job_map.add(name='',attr=attr,value=value, maptype='china',is_label_show=True)
job_map

In [56]:
from pyecharts import Geo

geo = Geo("全国主要城市程序员工资", "", title_color="#fff",
          title_pos="center", width=1000,
          height=600, background_color='#404a59')
attr, value = geo.cast(map_data)
geo.add("", attr, value, visual_range=[min(value), max(value)], maptype='china',visual_text_color="#fff", symbol_size=10,is_visualmap=True, is_label_show=True)

geo

In [ ]:
from pyecharts import Geo

geo = Geo("全国主要城市程序员工资", "", title_color="#fff",
          title_pos="center", width=1000,
          height=600, background_color='#404a59')
attr, value = geo.cast(map_data_headcount)

In [52]:

symbol_size = [v/1000 for v in value]
geo.add("", attr, value, visual_range=[min(value), max(value)], maptype='china',visual_text_color="#fff", symbol_size=10, is_visualmap=True, )
geo

In [31]:
symbol_size

[13.823,
 25.888,
 20.757,
 8.69,
 17.068,
 7.091,
 7.028,
 1.632,
 3.833,
 8.027,
 1.147,
 3.354,
 1.485,
 2.085,
 2.889,
 2.57,
 1.385,
 1.479,
 3.071,
 0.614,
 1.011,
 2.317,
 1.452,
 0.87]

In [103]:
data_city.sort_values(by='head_count', ascending=False).style.format({"percentage":"{:.2%}","salary_mean":"{:.0f}","salary_median":"{:.0f}"})

,city_,salary_mean,salary_median,head_count,percentage
17,shanghai,16626,15000,25888,18.55%
19,shenzhen,15631,14000,20757,14.87%
8,guangzhou,13032,12000,17068,12.23%
0,beijing,17453,15000,13823,9.90%
9,hangzhou,14977,12500,8690,6.23%
21,wuhan,11074,10417,8027,5.75%
14,nanjing,12874,12500,7091,5.08%
3,chengdu,11896,11500,7028,5.04%
22,xian,11075,10000,3833,2.75%
2,changsha,10847,10000,3354,2.40%


In [28]:
?geo.add

In [35]:
import plotly.plotly as py
import plotly.graph_objs as go

trace0 = go.Scatter(
    x=[1, 2, 3, 4],
    y=[10, 11, 12, 13],
    mode='markers',
    marker=dict(
        color=['rgb(93, 164, 214)', 'rgb(255, 144, 14)',
               'rgb(44, 160, 101)', 'rgb(255, 65, 54)'],
        opacity=[1, 0.8, 0.6, 0.4],
        size=[40, 60, 80, 100],
    )
)

data = [trace0]
py.iplot(data, filename='bubblechart-color')

ModuleNotFoundError: No module named 'plotly'